In [4]:
import pandas as pd
import numpy as np
import re
import emoji
import joblib
import torch
import string
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader, Dataset
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
train_data = pd.read_csv("train.csv")

In [7]:
import pandas as pd
dataset = pd.read_csv('train.csv')
print("Dataset shape:", dataset.shape)


Dataset shape: (40218, 2)


In [101]:
'''def remove_URL(text):
    text=str(text)
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)


def remove_emoji(text):
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'
        u'\U0001F300-\U0001F5FF'
        u'\U0001F680-\U0001F6FF'
        u'\U0001F1E0-\U0001F1FF'
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_mentions(text):
    ment = re.compile(r"(@[A-Za-z0-9]+)")
    return ment.sub(r'', text)


def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)


def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)'''

'def remove_URL(text):\n    text=str(text)\n    url = re.compile(r\'https?://\\S+|www\\.\\S+\')\n    return url.sub(r\'\', text)\n\n\ndef remove_emoji(text):\n    emoji_pattern = re.compile(\n        \'[\'\n        u\'😀-🙏\'\n        u\'🌀-🗿\'\n        u\'🚀-\U0001f6ff\'\n        u\'\U0001f1e0-🇿\'\n        u\'✂-➰\'\n        u\'Ⓜ-🉑\'\n        \']+\',\n        flags=re.UNICODE)\n    return emoji_pattern.sub(r\'\', text)\n\ndef remove_mentions(text):\n    ment = re.compile(r"(@[A-Za-z0-9]+)")\n    return ment.sub(r\'\', text)\n\n\ndef remove_html(text):\n    html = re.compile(r\'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});\')\n    return re.sub(html, \'\', text)\n\n\ndef remove_punct(text):\n    table = str.maketrans(\'\', \'\', string.punctuation)\n    return text.translate(table)'

In [103]:
import string
import urllib.parse
def remove_URL(text):
    text = str(text)
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

def retain_emoji(text):
    text = emoji.demojize(text, delimiters=("", " "))
    return text

def remove_mentions(text):
    ment = re.compile(r"(@[A-Za-z0-9_]+)")
    return ment.sub(r'', text)

def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)

def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

def preprocess_text(text):
    text = remove_URL(text)
    text = retain_emoji(text)
    text = remove_mentions(text)
    text = remove_html(text)
    text = remove_punct(text)
    return text.strip()

In [104]:
train_data['clean_text'] = train_data['tweets'].apply(lambda x: preprocess_text(x))
train_data['clean_text'] = train_data['clean_text'].apply(lambda x: x.lower())
cleaned = train_data['clean_text'].tolist()

for i,text in enumerate(cleaned):
    splits = text.split()
    splits = [word for word in splits if word not in set(nltk.corpus.stopwords.words('english'))]
    cleaned[i]=' '.join(splits)

train_data['clean_text']=cleaned

In [105]:
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_data['class'].values)

In [106]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_data['class'].values)


class_labels = train_data['class'].unique()


for label, encoded_label in zip(class_labels, label_encoder.transform(class_labels)):
    print(f"Original Label: {label}, Encoded Label: {encoded_label}")

Original Label: irony, Encoded Label: 0
Original Label: sarcasm, Encoded Label: 1


In [107]:

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data['tweets'].tolist(),
    train_data['class'].tolist(),
    test_size=0.2,
    random_state=42
)

print(f"\nTraining samples: {len(train_texts)}")
print(f"Validation samples: {len(val_texts)}")


Training samples: 32174
Validation samples: 8044


In [1]:
def scores(y_test, y_predicted):

  accuracy = accuracy_score(y_test, y_predicted)
  print('Accuracy: %f' % accuracy)

  precision = precision_score(y_test, y_predicted, average='weighted')
  print('Precision: %f' % precision)

  recall = recall_score(y_test, y_predicted, average='weighted')
  print('Recall: %f' % recall)

  f1 = f1_score(y_test, y_predicted, average='weighted')
  print('F1 score: %f' % f1)
  return precision, recall, f1

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
import nltk



nltk.download('stopwords')



train_data = pd.read_csv('train.csv')

x = train_data['tweets'].tolist()

stopWords = set(stopwords.words('english'))

filtered_tweets = []
for tweet in x:
    words = tweet.split()
    words_filtered = [word for word in words if word.lower() not in stopWords]
    filtered_tweets.append(" ".join(words_filtered))

vectorizer = CountVectorizer(lowercase=True)
features = vectorizer.fit_transform(filtered_tweets)

features_nd = features.toarray()

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(train_data['class'].values)

x_train, x_test, y_train, y_test = train_test_split(features_nd, y, train_size=0.80, random_state=1234)

print(x_train.shape)
print(y_train.shape)

log_model = LogisticRegression(max_iter=1000)
log_model.fit(x_train, y_train)

y_pred = log_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


dataset = pd.read_csv("/train.csv", sep=',')


print("Dataset columns:", dataset.columns)
print(dataset.head())


dataset = dataset.dropna(subset=['tweets', 'class'])


print(f"Dataset shape after cleaning: {dataset.shape}")


text_column_name = 'tweets'
label_column_name = 'class'


tokenizer = Tokenizer(num_words=30000, split=' ')
tokenizer.fit_on_texts(dataset[text_column_name].values)


X = tokenizer.texts_to_sequences(dataset[text_column_name].values)


X = pad_sequences(X, padding='post', maxlen=35)


print(f"Shape of tokenized and padded input features: {X.shape}")


Y = pd.get_dummies(dataset[label_column_name]).values


print(f"Shape of one-hot encoded labels: {Y.shape}")


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, random_state=42)


print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}")


train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))


train_dataset = train_dataset.shuffle(buffer_size=10000).batch(64)
test_dataset = test_dataset.batch(64)


model = tf.keras.Sequential([

    tf.keras.layers.Embedding(input_dim=30000, output_dim=128, input_length=35),


    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),


    tf.keras.layers.Dropout(0.5),


    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),


    tf.keras.layers.Dense(128, activation='relu'),


    tf.keras.layers.Dropout(0.5),


    tf.keras.layers.Dense(2, activation='softmax')
])


model.build(input_shape=(None, 35))


model.summary()


model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])


history = model.fit(train_dataset, epochs=10)


results = model.evaluate(test_dataset)
print('Test loss, test accuracy:', results)


y_pred = model.predict(X_test)


y_pred_classes = y_pred.argmax(axis=1)
y_test_classes = Y_test.argmax(axis=1)


print(f'Accuracy: {accuracy_score(y_test_classes, y_pred_classes)}')
print(f'Precision: {precision_score(y_test_classes, y_pred_classes, average="weighted")}')
print(f'Recall: {recall_score(y_test_classes, y_pred_classes, average="weighted")}')
print(f'F1 Score: {f1_score(y_test_classes, y_pred_classes, average="weighted")}')


Dataset columns: Index(['tweets', 'class'], dtype='object')
                                              tweets  class
0  Fav moment in Sepp Blatter vid (0:20): "We hav...  irony
1  Just found this while walking my human.... #ir...  irony
2  'Disrespected the wife of Prophet' - pseudo li...  irony
3  Do you know that super yeay satisfying feeling...  irony
4  If you're going to call someone ignorant and s...  irony
Dataset shape after cleaning: (40218, 2)
Shape of tokenized and padded input features: (40218, 35)
Shape of one-hot encoded labels: (40218, 2)
X_train shape: (32174, 35), Y_train shape: (32174, 2)
X_test shape: (8044, 35), Y_test shape: (8044, 2)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 35, 128)             │       3,840,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 35, 512)             │         788,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 35, 512)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 256)                 │         656,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,318,018 (20.29 MB)

 Trainable params: 5,318,018 (20.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
503/503 ━━━━━━━━━━━━━━━━━━━━ 370s 720ms/step - accuracy: 0.8354 - loss: 0.3017
Epoch 2/10
503/503 ━━━━━━━━━━━━━━━━━━━━ 361s 718ms/step - accuracy: 0.9992 - loss: 0.0035
Epoch 3/10
503/503 ━━━━━━━━━━━━━━━━━━━━ 383s 720ms/step - accuracy: 0.9990 - loss: 0.0038
Epoch 4/10
503/503 ━━━━━━━━━━━━━━━━━━━━ 379s 714ms/step - accuracy: 0.9999 - loss: 7.8646e-04
Epoch 5/10
503/503 ━━━━━━━━━━━━━━━━━━━━ 359s 713ms/step - accuracy: 0.9997 - loss: 0.0015
Epoch 6/10
503/503 ━━━━━━━━━━━━━━━━━━━━ 383s 715ms/step - accuracy: 0.9999 - loss: 4.4719e-04
Epoch 7/10
503/503 ━━━━━━━━━━━━━━━━━━━━ 378s 708ms/step - accuracy: 0.9998 - loss: 0.0012
Epoch 8/10
503/503 ━━━━━━━━━━━━━━━━━━━━ 392s 728ms/step - accuracy: 0.9997 - loss: 0.0015
Epoch 9/10
503/503 ━━━━━━━━━━━━━━━━━━━━ 380s 724ms/step - accuracy: 0.9999 - loss: 3.8231e-04
Epoch 10/10
503/503 ━━━━━━━━━━━━━━━━━━━━ 382s 725ms/step - accuracy: 0.9997 - loss: 7.1929e-04
126/126 ━━━━━━━━━━━━━━━━━━━━ 26s 201ms/step - accuracy: 0.9986 - loss: 0.0157
Test 

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences




sentences = [
    "I love getting mail saying that my medical &amp; personal information has been compromised. #sarcasm",
    "This has been the worst experience of my life.",
    "Oh great, another Monday morning. Just what I needed.",
    "The fire station burned down last night.",
    "I went to the store to buy some groceries."
]


tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')


predictions = model.predict(padded_sequences)


class_labels = ['irony', 'sarcasm']


print("Prediction Results:")
for i, sentence in enumerate(sentences):
    predicted_class_index = np.argmax(predictions[i])
    predicted_class = class_labels[predicted_class_index]
    print(f"Input Sentence: {sentence}")
    print(f"Prediction: {predicted_class} (Probabilities: {predictions[i]})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 723ms/step
Prediction Results:
Input Sentence: I love getting mail saying that my medical &amp; personal information has been compromised. #sarcasm
Prediction: irony (Probabilities: [9.9968445e-01 3.1557016e-04])
Input Sentence: This has been the worst experience of my life.
Prediction: irony (Probabilities: [1.0000000e+00 1.2935479e-08])
Input Sentence: Oh great, another Monday morning. Just what I needed.
Prediction: sarcasm (Probabilities: [5.2503660e-06 9.9999475e-01])
Input Sentence: The fire station burned down last night.
Prediction: irony (Probabilities: [1.0000000e+00 1.5658651e-08])
Input Sentence: I went to the store to buy some groceries.
Prediction: irony (Probabilities: [9.9997509e-01 2.4897232e-05])


In [ ]:
import os


save_dir = "/content/saved_models"


if not os.path.exists(save_dir):
    os.makedirs(save_dir)


model_path = os.path.join(save_dir, "sarcasm_and_irony_model.h5")


model.save(model_path)

print(f"Model saved at: {model_path}")


Model saved at: /content/saved_models/sarcasm_and_irony_model.h5
